In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from functools import partial
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.distributions as dist
import pyro
import pyro.distributions as pyro_dist
from pyro.infer.autoguide import AutoDiagonalNormal, AutoMultivariateNormal, init_to_mean, AutoNormal
from pyro.infer import SVI, Trace_ELBO

import bayesreg_pyro as br
import vi_diagnostics as vid

In [12]:
data = pd.read_csv("bodyfat.csv").drop("Density", axis=1)
data = data - data.mean()

y_data = torch.Tensor(data["Bodyfat"].values)
x_data = torch.Tensor(data.drop("Bodyfat", axis=1).values)
in_features = x_data.shape[1]

In [15]:
model = br.BayesianRegression(in_features, 1)
guide = AutoDiagonalNormal(model) # Stochastic Mean field
adam = pyro.optim.Adam({"lr": 0.01})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

torch.manual_seed(12)
pyro.clear_param_store()
tic=time.time()
elbos, samples = br.run_vi(x_data, y_data, svi, model, guide)
toc=time.time()
print(f"SVI time: {toc-tic} sec")

SVI time: 19.9405779838562 sec


In [ ]:
# I think we can use much less samples as optimiser is already in a "good region"?
inf_engine = partial(
    br.run_vi, vi_engine=svi, model=model, guide=guide, num_iterations=5000, return_elbo=False
)
torch.manual_seed(1212)
tic = time.time()
prob = vid.diagnose_vi_VSBC(x_data, model, guide, inf_engine, replications=150)
toc = time.time()


In [47]:
print(f"2 replications took {toc-tic}sec")

2 replications took 11.39851188659668sec


In [49]:
prob

array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 9.9999970e-01, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 9.9965870e-01, 1.9035935e-03, 0.0000000e+00,
        3.6424398e-04, 1.1244416e-04, 2.8929168e-01]], dtype=float32)